In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: alexisjihyeross. Use `wandb login --relogin` to force relogin


True

In [ ]:
%wandb alexisjihyeross/0131

In [2]:
import pandas as pd 
import wandb
import numpy as np

api = wandb.Api(timeout=50)
entity, project = "lm-informants", "lm-informants_maybefinal_three"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [3]:
df = pd.DataFrame()

In [9]:
from tqdm import tqdm

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run in tqdm(runs): 
    
    # add all the keys that are logged that you want to download
    keys = [
        "step",
        "auc",
        ]
                
    df = pd.DataFrame()
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
            
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # if key doesn't exist, set to nan (though this shouldn't happen)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})
    
    exp.update({f"config/{key}": val for key, val in config.items()})
    exp.update({f"strategy": run.name})

    all_data.append(exp)
    
    
mega_df = pd.DataFrame(all_data)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.51it/s]


In [10]:
display(mega_df)

,step,auc,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,config/feature_type,config/lexicon_file,config/converge_type,config/log_log_alpha_ratio,strategy
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7576025987493322, 0.759...",0,0,kl,0.001,0.1,True,atr_harmony,None,symmetric,2,kl
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7576025987493322, 0.759...",0,0,eig,0.001,0.1,True,atr_harmony,None,symmetric,2,eig
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7524706972944096, 0.754...",9,0,kl_train_history,0.001,0.5,True,atr_harmony,None,symmetric,1,kl_train_history
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7524706972944096, 0.750...",9,0,kl_train_mixed,0.001,0.5,True,atr_harmony,None,symmetric,1,kl_train_mixed
4,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7524706972944096, 0.750...",9,0,eig_train_mixed,0.001,0.5,True,atr_harmony,None,symmetric,1,eig_train_mixed
5,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7551161895484397, 0.755...",9,0,eig_train_history,0.001,0.5,True,atr_harmony,None,symmetric,1,eig_train_history
6,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7524706972944096, 0.750...",9,0,eig_train_model,0.001,0.5,True,atr_harmony,None,symmetric,1,eig_train_model
7,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7524706972944096, 0.750...",9,0,kl_train_model,0.001,0.5,True,atr_harmony,None,symmetric,1,kl_train_model
8,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7562592307450586, 0.756...",8,0,kl_train_history,0.001,0.5,True,atr_harmony,None,symmetric,1,kl_train_history
9,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.7524706972944096, 0.7562592307450586, 0.756...",8,0,kl_train_mixed,0.001,0.5,True,atr_harmony,None,symmetric,1,kl_train_mixed


In [11]:
out_path = './runs.csv'
mega_df.to_csv(out_path)